In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("finalRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows
rl.head()


Metadata file is not present in the given path; proceeding to read the csv file.


,id,name,genus,family,ecology,countries,threat_paragraph,conservation_paragraph,pop_trend,status,country_count
0,0,ochlockonee moccasinshell,Medionidus,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,1,nelson's spiny pocket mouse,Heteromys,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the significant forest loss in its habitat. Habitat fra...,"['\n This species occurs in a newly-named national park, Tacana, in Mexico.\n\n \n ']",Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n",2
2,2,comoro friar,Amauris,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,3,"atlantic halibut, halibut",Hippoglossus,Pleuronectidae,NaN,<div>Atlantic å_åÑåÐ northeast; Atlantic å_åÑåÐ northwest</div>; Canada (Newfoundland I); Denmar...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,10,oktibbeha riverlet crayfish,Hobbseus,Cambaridae,Freshwater,United States (Mississippi);,\n This species is threatened by habitat degradation through residential and urban developmen...,"['\n', <span style=""font-style: italic;""></span>, 'This species has been given the Global Herita...",Unknown,\n Endangered\n\n\n B1ab(iii)\n\n,1


In [2]:
# CME
em.get_key(rl) # get key

# Note: The id column of the  redlist csv doesn't actually look like the object created here (rl).  In the csv,
# id numbers jump around since they represent original row number of the original scraped data csv.  Apparently
# this library can detect what we want and enumerate things in order if you give it a key feature like this.


'id'

In [3]:
#Read in ARKIVE
# Read in csv as dataframe
ar = em.read_csv_metadata("finalArkive.csv", encoding="ISO-8859-1", key="id")

# glance at first few rows
ar.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,id,name,scientific_name,nickname,common_name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threats_keywords,conservation_keywords,tCount,cCount
0,0,alcatraz de abbott; piquero de abbott; abbott's booby,Alcatraz de Abbott; Piquero de Abbott,Papasula abbotti,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula,Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0
1,1,abbotí_¢ëí_ëés duiker; abbottí_¢ëí_ëés duiker,Abbotí_¢ëí_ëés duiker,Cephalophus spadix,Abbottí_¢ëí_ëés duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50 í_¢ëí_ëí± 60 kg (2),Like many other duikers Abbott\xe2\x80\x99s duiker is being impacted by a combination of hunting...,Although Abbott\xe2\x80\x99s duiker occurs within several protected areas such as the Kilimanjar...,hunting;,protected;,1,1
2,2,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Adelaide pigmy blue-tongue skink; pygmy bluetongue,Tiliqua adelaidensis,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua,Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australia\xe2\x80\x99s ap...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1
3,3,adriatic trout; softmouth trout; adriatic salmon,Adriatic trout; softmouth trout,Salmo obtusirostris,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo,80 - 425 g (4),The Adriatic salmon faces a number of threats including overfishing for sport and for food as we...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,fishing;overfishing;invasive;pet;,,4,0
4,4,black-footed penguin; jackass penguin; african penguin,black-footed penguin; jackass penguin,Spheniscus demersus,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus,Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2


In [4]:
# Remove parentheticals from arkive common names
# author: CME
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['common_name'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'common_name'] = noparens
    else:
        error = error + 1

print(error)


818


In [5]:
# Concatenate nickname and common_name fields in arkive
# author: @andrewedstrom

# remove parenthesis contents
nameColumn = []
for index, row in ar.iterrows():
    nickname = str(row['scientific_name']) #look here if errors start happening
    nickname = re.sub('[^A-Za-z0-9;,\s]+', '', nickname).lower()
    common = str(row['common_name'])
    common = re.sub('[^A-Za-z0-9;,\s]+', '', common).lower()
    if common in nickname: 
        nameColumn.append(nickname)
    else:
        nameColumn.append((nickname + "; " + common))
ar['name'] = nameColumn
ar.head()

,id,name,scientific_name,nickname,common_name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threats_keywords,conservation_keywords,tCount,cCount
0,0,alcatraz de abbott; piquero de abbott; abbotts booby,Alcatraz de Abbott; Piquero de Abbott,Papasula abbotti,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula,Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0
1,1,abbots duiker; abbotts duiker,Abbotí_¢ëí_ëés duiker,Cephalophus spadix,Abbottí_¢ëí_ëés duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50 í_¢ëí_ëí± 60 kg (2),Like many other duikers Abbott\xe2\x80\x99s duiker is being impacted by a combination of hunting...,Although Abbott\xe2\x80\x99s duiker occurs within several protected areas such as the Kilimanjar...,hunting;,protected;,1,1
2,2,adelaide pigmy bluetongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Adelaide pigmy blue-tongue skink; pygmy bluetongue,Tiliqua adelaidensis,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua,Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australia\xe2\x80\x99s ap...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1
3,3,adriatic trout; softmouth trout; adriatic salmon,Adriatic trout; softmouth trout,Salmo obtusirostris,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo,80 - 425 g (4),The Adriatic salmon faces a number of threats including overfishing for sport and for food as we...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,fishing;overfishing;invasive;pet;,,4,0
4,4,blackfooted penguin; jackass penguin; african penguin,black-footed penguin; jackass penguin,Spheniscus demersus,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus,Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2


In [6]:
# Lowercase redlist name field
# author: @andrewedstrom
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in rl.iterrows():
    n = str(row['name'])
    n = re.sub('[^A-Za-z0-9;,\s]+', '', n).lower()

    rl.loc[index, 'name'] = n.lower()

rl.head()

,id,name,genus,family,ecology,countries,threat_paragraph,conservation_paragraph,pop_trend,status,country_count
0,0,ochlockonee moccasinshell,Medionidus,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,1,nelsons spiny pocket mouse,Heteromys,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the significant forest loss in its habitat. Habitat fra...,"['\n This species occurs in a newly-named national park, Tacana, in Mexico.\n\n \n ']",Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n",2
2,2,comoro friar,Amauris,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,3,"atlantic halibut, halibut",Hippoglossus,Pleuronectidae,NaN,<div>Atlantic å_åÑåÐ northeast; Atlantic å_åÑåÐ northwest</div>; Canada (Newfoundland I); Denmar...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,10,oktibbeha riverlet crayfish,Hobbseus,Cambaridae,Freshwater,United States (Mississippi);,\n This species is threatened by habitat degradation through residential and urban developmen...,"['\n', <span style=""font-style: italic;""></span>, 'This species has been given the Global Herita...",Unknown,\n Endangered\n\n\n B1ab(iii)\n\n,1


In [7]:
# Remove parentheticals from arkive genus
# author: @andrewedstrom
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['genus'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'genus'] = noparens
    else:
        error = error + 1

print(error)

818


In [8]:
#rl.to_csv('finalRedlist.csv')
#ar.to_csv('finalArkive.csv')

In [15]:
# Rule-based blocking over input tables
# first get features that can be used
feature_table = em.get_features_for_blocking(rl, ar)


In [16]:
#len(feature_table)
feature_table
#em.get_attr_corres(rl, ar)['ltable']

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x1159ac1e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x1159ac158>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x115ba2d08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x115ba2950>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,family_family_jac_qgm_3_qgm_3,family,family,qgm_3,qgm_3,jaccard,<function family_family_jac_qgm_3_qgm_3 at 0x115ba2378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,family_family_cos_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,cosine,<function family_family_cos_dlm_dc0_dlm_dc0 at 0x115ba2a60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,family_family_jac_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,jaccard,<function family_family_jac_dlm_dc0_dlm_dc0 at 0x115ba2bf8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,family_family_mel,family,family,None,None,monge_elkan,<function family_family_mel at 0x1160ba7b8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,family_family_lev_dist,family,family,None,None,lev_dist,<function family_family_lev_dist at 0x1160baf28>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,family_family_lev_sim,family,family,None,None,lev_sim,<function family_family_lev_sim at 0x1160baea0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [11]:
# CME -- I'm realizing now that we may want to start with the attribute equivalence
# blocker for names and see where that leaves us.  On my TO-DO.
equivB = em.AttrEquivalenceBlocker()
C0 = equivB.block_tables(rl, ar, 'name', 'name',
                    l_output_attrs=['name', 'genus'], 
                    r_output_attrs=['name', 'genus'])
C0

# lolwat this reveals zero matches?


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus


In [17]:
#Do overlap blocking on name to get candidate set
#Perform some kind of blocking to find candidate pairs
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'name', 'name', 
                     l_output_attrs=['name', 'genus', 'family'], 
                     r_output_attrs=['name', 'genus', 'family'], overlap_size=2)
C0

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus
0,0,1470,2,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,adelaide pigmy bluetongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
1,1,1192,3,adriatic salmon,Salmo,adriatic trout; softmouth trout; adriatic salmon,Salmo
2,2,2017,5,"african teak, afromosia, afrormosia",Pericopsis,afrormosia; assamela; kokrodua; african teak,Pericopsis
3,3,1515,8,"andes smooth frog, lake junin frog, marbled water frog",Telmatobius,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
4,4,4977,8,hunaco water frog,Telmatobius,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
5,5,7524,8,titicaca water frog,Telmatobius,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
6,6,4317,8,efulen water frog,Petropedetes,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
7,7,4318,8,perrets water frog,Petropedetes,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
8,8,4363,8,albanian water frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
9,9,5807,8,arico water frog,Telmatobius,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax


In [28]:
#Only keep candidate pairs which share genus
ab = em.AttrEquivalenceBlocker()
C1 = ob.block_candset(C0, 'genus', 'genus')
C1

#Do overlap blocking on name to get candidate set
#Perform some kind of blocking to find candidate pairs
#C1 = ab.block_tables(rl, ar,'genus', 'genus', 
#                     l_output_attrs=['name', 'genus'], 
#                     r_output_attrs=['name', 'genus'])


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus
0,0,1470,2,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,adelaide pigmy bluetongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
1,1,1192,3,adriatic salmon,Salmo,adriatic trout; softmouth trout; adriatic salmon,Salmo
2,2,2017,5,"african teak, afromosia, afrormosia",Pericopsis,afrormosia; assamela; kokrodua; african teak,Pericopsis
8,8,4363,8,albanian water frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
16,16,4363,9,albanian water frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
21,21,339,12,blind swamp eel,Ophisternon,blind swamp eel; anguila ciega,Ophisternon
22,22,661,13,decarys leaf chameleon,Brookesia,armoured chameleon; stump tailed chameleon; antsingy leaf chameleon,Brookesia
23,23,4575,16,aran rock lizard,Iberolacerta,lagartija aranesa; aran rock lizard,Iberolacerta
24,24,4576,16,aurelios rock lizard,Iberolacerta,lagartija aranesa; aran rock lizard,Iberolacerta
25,25,82,21,"asian giant tortoise, asian tortoise, black giant tortoise, burmese brown tortoise, burmese moun...",Manouria,asian brown tortoise; asian giant tortoise; black giant tortoise; burmese brown tortoise; burmes...,Manouria


In [31]:
C2 = equivB.block_tables(rl, ar, 'family', 'family',
                    l_output_attrs=['name', 'genus' ,'family'], 
                    r_output_attrs=['name', 'genus', 'family'])

C3 = ob.block_candset(C2, 'genus', 'genus')
C3


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,rtable_name,rtable_genus,rtable_family
25,25,705,589,oval pigtoe,Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
91,91,5293,706,"orangefooted pimpleback mussel, orangefoot pimpleback",Plethobasus,Unionidae,orangefooted pearlymussel; orangefooted pimpleback mussel; orangefoot pimpleback,Plethobasus,Unionidae
93,93,5296,589,"club naiad, clubshell, clubshell pearly mussel",Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
97,97,5297,589,"flat pigtoe, marshalls mussel",Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
101,101,5298,589,"rough pigtoe, rough pigtoe pearly mussel",Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
105,105,5299,589,painted clubshell,Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
109,109,5300,589,james river spinymussel,Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
113,113,5301,589,dark pigtoe,Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
117,117,5302,589,southern pigtoe,Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae
121,121,5303,589,cumberland pigtoe,Pleurobema,Unionidae,club naiad; clubshell; northern clubshell; clubshell pearly mussel,Pleurobema,Unionidae


In [33]:
C = em.combine_blocker_outputs_via_union([C1, C3])
C

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,rtable_name,rtable_genus,rtable_family
0,0,12,126,"danube salmon, huchen",Hucho,Salmonidae,huchen; danube salmon,Hucho,Salmonidae
1,1,20,285,"malagasy giant jumping rat, malagasy giant rat",Hypogeomys,Nesomyidae,malagasy giant jumping rat; malagasy giant rat,Hypogeomys,Nesomyidae
2,2,22,143,"elongated tortoise, pineapple tortoise, rednosed tortoise, yellowheaded tortoise, yellow tortoise",Indotestudo,Testudinidae,pineapple tortoise; rednosed tortoise; yellow tortoise; yellowheaded tortoise; elongated tortoise,Indotestudo,Testudinidae
3,3,22,159,"elongated tortoise, pineapple tortoise, rednosed tortoise, yellowheaded tortoise, yellow tortoise",Indotestudo,Testudinidae,celebes tortoise; forstens tortoise,Indotestudo,Testudinidae
4,4,23,143,"celebes tortoise, forstens tortoise, travancore tortoise",Indotestudo,Testudinidae,pineapple tortoise; rednosed tortoise; yellow tortoise; yellowheaded tortoise; elongated tortoise,Indotestudo,Testudinidae
5,5,23,159,"celebes tortoise, forstens tortoise, travancore tortoise",Indotestudo,Testudinidae,celebes tortoise; forstens tortoise,Indotestudo,Testudinidae
6,6,48,320,"nelsons antelope squirrel, san joaquin antelope ground squirrel, san joaquin antelope squirrel",Ammospermophilus,Sciuridae,san joaquin antelope ground squirrel; san joaquin antelope squirrel; nelsons antelope squirrel,Ammospermophilus,Sciuridae
7,7,53,702,milneedwardss sportive lemur,Lepilemur,Lepilemuridae,lemur comadreja septentrional; northern sportive lemur,Lepilemur,Lepilemuridae
8,8,54,702,"drybush weasel lemur, whitefooted sportive lemur",Lepilemur,Lepilemuridae,lemur comadreja septentrional; northern sportive lemur,Lepilemur,Lepilemuridae
9,9,55,702,"lightnecked sportive lemur, microdon sportive lemur, smalltoothed sportive lemur",Lepilemur,Lepilemuridae,lemur comadreja septentrional; northern sportive lemur,Lepilemur,Lepilemuridae


In [34]:
em.to_csv_metadata(C, './candidate_set.csv')

True

In [36]:
C = em.read_csv_metadata('candidate_set.csv', ltable=rl, rtable=ar)

In [37]:
C

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,rtable_name,rtable_genus,rtable_family
0,0,12,126,"danube salmon, huchen",Hucho,Salmonidae,huchen; danube salmon,Hucho,Salmonidae
1,1,20,285,"malagasy giant jumping rat, malagasy giant rat",Hypogeomys,Nesomyidae,malagasy giant jumping rat; malagasy giant rat,Hypogeomys,Nesomyidae
2,2,22,143,"elongated tortoise, pineapple tortoise, rednosed tortoise, yellowheaded tortoise, yellow tortoise",Indotestudo,Testudinidae,pineapple tortoise; rednosed tortoise; yellow tortoise; yellowheaded tortoise; elongated tortoise,Indotestudo,Testudinidae
3,3,22,159,"elongated tortoise, pineapple tortoise, rednosed tortoise, yellowheaded tortoise, yellow tortoise",Indotestudo,Testudinidae,celebes tortoise; forstens tortoise,Indotestudo,Testudinidae
4,4,23,143,"celebes tortoise, forstens tortoise, travancore tortoise",Indotestudo,Testudinidae,pineapple tortoise; rednosed tortoise; yellow tortoise; yellowheaded tortoise; elongated tortoise,Indotestudo,Testudinidae
5,5,23,159,"celebes tortoise, forstens tortoise, travancore tortoise",Indotestudo,Testudinidae,celebes tortoise; forstens tortoise,Indotestudo,Testudinidae
6,6,48,320,"nelsons antelope squirrel, san joaquin antelope ground squirrel, san joaquin antelope squirrel",Ammospermophilus,Sciuridae,san joaquin antelope ground squirrel; san joaquin antelope squirrel; nelsons antelope squirrel,Ammospermophilus,Sciuridae
7,7,53,702,milneedwardss sportive lemur,Lepilemur,Lepilemuridae,lemur comadreja septentrional; northern sportive lemur,Lepilemur,Lepilemuridae
8,8,54,702,"drybush weasel lemur, whitefooted sportive lemur",Lepilemur,Lepilemuridae,lemur comadreja septentrional; northern sportive lemur,Lepilemur,Lepilemuridae
9,9,55,702,"lightnecked sportive lemur, microdon sportive lemur, smalltoothed sportive lemur",Lepilemur,Lepilemuridae,lemur comadreja septentrional; northern sportive lemur,Lepilemur,Lepilemuridae
